### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d
from tqdm.notebook import tqdm
from neuropy.core import Epoch
from neuropy.utils.mathutil import min_max_scaler
import pandas as pd

### Example radon transform figures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d
from neuropy.analyses.decoders import radon_transform
from hfuncs import radon_transform_gpu
from time import time

s = time()
for i in range(100):
    radon_transform(np.ones((50, 10)))
e = time()
print(e - s)

s = time()
for i in range(5000):
    radon_transform_gpu(np.ones((50, 10)))
e = time()
print(e - s)


# sess = subjects.nsd.ratUday2[0]

# maze = sess.paradigm["maze"].flatten()
# post = sess.paradigm["post"].flatten()
# neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
# pos = sess.maze
# # pos.t_start = pos.t_start - 0.5
# run = sess.run
# pf = Pf1D(
#     neurons=neurons,
#     position=pos,
#     speed_thresh=4,
#     sigma=4,
#     grid_bin=2,
#     epochs=run,
#     frate_thresh=1,
# )
# # np.random.default_rng().shuffle(pf.tuning_curves)
# pf_neurons = neurons.get_by_id(pf.neuron_ids)
# epochs = sess.pbe.time_slice(maze[0], maze[0]+500)
# decode = Decode1d(
#     neurons=pf_neurons, ratemap=pf, epochs=epochs, bin_size=0.02
# )


In [ ]:
import cupy as cp
from numpy.random import default_rng
from neuropy.analyses.decoders import radon_transform
from hfuncs import radon_transform_gpu
from neuropy import plotting
from scipy import stats
from neuropy.utils.mathutil import min_max_scaler
import time
from joblib import Parallel, delayed

n_posteriors = len(decode.posterior)
posterior_ind = default_rng().integers(0, n_posteriors, n_posteriors)
# posterior_ind = [382, 42, 131, 162, 287, 284, 127, 331, 294, 205]
# posterior_ind =[189, 139, 180, 178, 188, 103, 158,  15, 123, 156]
# posterior_ind= [ 7, 126, 114, 116, 115]
arrs = [decode.posterior[i] for i in posterior_ind]
len_arr = np.cumsum([_.shape[1] for _ in arrs])[:-1]
zsc_tuning = stats.zscore(pf.tuning_curves, axis=1)
sort_ind = np.argsort(np.argmax(zsc_tuning, axis=1))
post_gpu = cp.asarray(np.hstack(arrs))

s = time.time()
arrs_gpu = cp.hsplit(post_gpu, len_arr)
for i, arr in enumerate(arrs_gpu):

    score, velocity, intercept = radon_transform_gpu(arr, dt=0.02, dx=2, neighbours=4)

e = time.time()
print(e - s)

# s = time.time()
# for i, arr in enumerate(arrs):
#     t_start = epochs[posterior_ind[i]].flatten()[0]
#     score, velocity, intercept = radon_transform(arr, dt=0.02, dx=2, neighbours=4)
# e = time.time()
# print(e-s)

s = time.time()
results = Parallel(n_jobs=5)(
    delayed(radon_transform)(
        epoch,
        dt=0.02,
        dx=2,
        neighbours=4,
    )
    for epoch in arrs
)
score, velocity, intercept = np.asarray(results).T
e = time.time()
print(e - s)


In [ ]:
%matplotlib widget
from numpy.random import default_rng
from neuropy.analyses.decoders import radon_transform_gpu,radon_transform
from neuropy import plotting
from scipy import stats
from neuropy.utils.mathutil import min_max_scaler
import time

n_posteriors = len(decode.posterior)
posterior_ind = default_rng().integers(0,n_posteriors,5)
# posterior_ind = [382, 42, 131, 162, 287, 284, 127, 331, 294, 205]
# posterior_ind =[189, 139, 180, 178, 188, 103, 158,  15, 123, 156]
# posterior_ind= [ 7, 126, 114, 116, 115]
arrs = [decode.posterior[i] for i in posterior_ind]

_, axs = plt.subplots(2, 5,sharey='row',sharex='col')
# axs = axs.reshape(-1)
zsc_tuning = stats.zscore(pf.tuning_curves,axis=1)
sort_ind = np.argsort(np.argmax(zsc_tuning,axis=1))

s = time.time()
for i, arr in enumerate(arrs):
    # arr = min_max_scaler(arr,axis=0)
    t_start = epochs[posterior_ind[i]].flatten()[0]

    score, velocity, intercept = radon_transform_gpu(arr, dt=0.02, dx=2, neighbours=4)
    # arr = np.apply_along_axis(np.convolve, axis=0, arr=arr, v=np.ones(2 * 4 + 1))
    # t = np.arange(arr.shape[1]) * 0.02+t_start
    # pos = np.arange(arr.shape[0]) * 2

    # axs[0,i].pcolormesh(t, pos,arr, cmap="hot")
    # axs[0,i].plot(t, velocity * (t-t_start) + intercept, color="w")
    # axs[0,i].set_ylim([pos.min(), pos.max()])
    # axs[0,i].set_title(score.round(2))

    # plotting.plot_raster(pf_neurons[sort_ind].time_slice(t_start=t[0],t_stop=t[-1]),ax=axs[1,i],color='k')
e = time.time()
print(e-s)

s = time.time()
for i, arr in enumerate(arrs):
    # arr = min_max_scaler(arr,axis=0)
    t_start = epochs[posterior_ind[i]].flatten()[0]

    score, velocity, intercept = radon_transform(arr, dt=0.02, dx=2, neighbours=4)
    # arr = np.apply_along_axis(np.convolve, axis=0, arr=arr, v=np.ones(2 * 4 + 1))
    # t = np.arange(arr.shape[1]) * 0.02+t_start
    # pos = np.arange(arr.shape[0]) * 2

    # axs[0,i].pcolormesh(t, pos,arr, cmap="hot")
    # axs[0,i].plot(t, velocity * (t-t_start) + intercept, color="w")
    # axs[0,i].set_ylim([pos.min(), pos.max()])
    # axs[0,i].set_title(score.round(2))

    # plotting.plot_raster(pf_neurons[sort_ind].time_slice(t_start=t[0],t_stop=t[-1]),ax=axs[1,i],color='k')
e = time.time()
print(e-s)


### Visualize decoding in one session

In [ ]:
sessions = subjects.nsd.ratUday2

In [ ]:
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # pos.t_start = pos.t_start - 0.5
    forward = sess.run["backward"]

    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=5,
        sigma=4,
        grid_bin=2,
        epochs=forward,
        frate_thresh=0.1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    epochs = sess.pbe.time_slice(post[0], post[0] + 2 * 3600)
    decode = Decode1d(
        neurons=pf_neurons,
        ratemap=pf,
        epochs=epochs,
        bin_size=0.02,
        score_method="wcorr",
    )
    decode.calculate_shuffle_score(n_iter=1000, method="neuron_id")


In [ ]:
_,ax = plt.subplots()

bins = np.arange(-1,1,0.1)
hist_percentile = np.histogram(decode.score,bins)[0]
ax.violinplot(decode.score)

In [ ]:
decode.plot_summary()

In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

measure = decode.percentile_score 
measure[np.isnan(measure)] = 0
# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
bplot.output_notebook()
p = plot_in_bokeh(
    x=epochs.starts / 3600,
    y=measure,
    img_arr=decode.posterior,
    color_by=measure,
    palette="jet",
    size=10,
)
# p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)


### Replay score distribution comparison in POST

In [ ]:
sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

In [ ]:
score_df = []
dist_replay_df = []
for sub, sess in enumerate(tqdm(sessions)):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # run = sess.run
    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=5,
        sigma=4,
        grid_bin=2,
        # epochs=run,
        frate_thresh=1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    zts = np.array([0, 1, 2, 4, 5, 7])
    starts = zts * 3600 + post[0]

    epochs = []
    for s, zt in zip(starts, zts):
        epochs.append(
            sess.pbe.time_slice(s, s + 3600).set_labels(f"{zt+1}").to_dataframe()
        )

    epochs = Epoch(pd.concat(epochs, ignore_index=True))

    decode = Decode1d(
        neurons=pf_neurons,
        ratemap=pf,
        epochs=epochs,
        bin_size=0.02,
        decode_margin=15,
        nlines=5000,
    )
    jump_distance = [np.abs(np.diff(_)).mean() for _ in decode.decoded_position]
    weighted_corr = decode.weighted_correlation

    norm_pos = min_max_scaler(decode.ratemap.xbin_centers)
    decoded_position_mean = np.nanmean(np.hstack(decode.posterior), axis=1)
    pos_bins = np.linspace(0, 1, 50)
    mean_pos = np.interp(pos_bins, norm_pos, decoded_position_mean)

    df_dist = pd.DataFrame(
        dict(
            bins=pos_bins,
            mean_pos=mean_pos,
            name=sess.animal.name,
            grp=sess.tag,
        )
    )

    df = pd.DataFrame(
        dict(
            score=decode.score,
            velocity=decode.velocity,
            jump_distance=jump_distance,
            weighted_corr=weighted_corr,
            epoch=epochs.labels,
            name=sess.animal.name,
            grp=sess.tag,
        )
    )
    score_df.append(df)
    dist_replay_df.append(df_dist)

score_df = pd.concat(score_df, ignore_index=True)
dist_replay_df = pd.concat(dist_replay_df, ignore_index=True)

subjects.GroupData().save(score_df, "replay_post_score")
subjects.GroupData().save(dist_replay_df, "replay_pos_distribution")


In [ ]:
score_df = pd.concat(score_df, ignore_index=True)
dist_replay_df = pd.concat(dist_replay_df, ignore_index=True)

subjects.GroupData().save(score_df, "replay_post_score")
subjects.GroupData().save(dist_replay_df, "replay_pos_distribution")



In [ ]:
import seaborn as sns
_,ax = plt.subplots()
sns.violinplot(
    data=score_df,
    x="epoch",
    y="score",
    hue="grp",
    split=True,
    inner=None,
    scale="width",
)


### scratch code for pooled randon transform

In [ ]:
nbins = [_.shape[1] for _ in decode.posterior[:60]]
post = np.hstack(decode.posterior[:60])
arr = post
nlines = 10000
dt = 1
dx = 1
neighbours = 1
t = np.concatenate([np.arange(0, _) for _ in nbins])
max_t = max(nbins)
nt = len(t)
tmid = (nt + 1) / 2 - 1

pos = np.arange(arr.shape[0])
npos = len(pos)
pmid = (npos + 1) / 2 - 1

# using convolution to sum neighbours
arr = np.apply_along_axis(
    np.convolve, axis=0, arr=arr, v=np.ones(2 * neighbours + 1), mode="same"
)

# exclude stationary events by choosing phi little below 90 degree
# NOTE: angle of line is given by (90-phi), refer Kloosterman 2012
phi = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=nlines)
diag_len = np.sqrt((max_t - 1) ** 2 + (npos - 1) ** 2)
rho = np.random.uniform(low=-diag_len / 2, high=diag_len / 2, size=nlines)

rho_mat = np.tile(rho, (nt, 1)).T
phi_mat = np.tile(phi, (nt, 1)).T
t_mat = np.tile(t, (nlines, 1))
posterior = np.zeros((nlines, nt))

y_line = ((rho_mat - (t_mat - tmid) * np.cos(phi_mat)) / np.sin(phi_mat)) + pmid
y_line = np.rint(y_line).astype("int")

# if line falls outside of array in a given bin, replace that with median posterior value of that bin across all positions
t_out = np.where((y_line < 0) | (y_line > npos - 1))
t_in = np.where((y_line >= 0) & (y_line <= npos - 1))
posterior[t_out] = np.median(arr[:, t_out[1]], axis=0)
posterior[t_in] = arr[y_line[t_in], t_in[1]]

# old_settings = np.seterr(all="ignore")
posterior_mean = np.nanmean(posterior, axis=1)

best_line = np.argmax(posterior_mean)
score = posterior_mean[best_line]
best_phi, best_rho = phi[best_line], rho[best_line]
time_mid, pos_mid = nt * dt / 2, npos * dx / 2

velocity = dx / (dt * np.tan(best_phi))
intercept = (
    (dx * time_mid) / (dt * np.tan(best_phi))
    + (best_rho / np.sin(best_phi)) * dx
    + pos_mid
)
# np.seterr(**old_settings)

# return score, -velocity, intercept
